In [1]:
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index     # chỉ số thuộc tính chọn để chia tại node này
        self.threshold = threshold             # giá trị ngưỡng để chia tại node này
        self.left = left                       # node con bên trái
        self.right = right                     # node con bên phải
        self.value = value                     # giá trị dự đoán tại node lá (nếu node này là lá)


In [2]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        p = counts / counts.sum()
        return -np.sum(p * np.log2(p))

    def _best_split(self, X, y):
        m = y.size
        if m <= 1:
            return None, None
        parent_entropy = self._entropy(y)
        best_info_gain = -1
        best_idx, best_thr = None, None
        for idx in range(self.n_features_):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = [np.sum(y == c) for c in range(self.n_classes_)]
            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                if thresholds[i] == thresholds[i - 1]:
                    continue
                p_left = np.sum(num_left) / i
                p_right = np.sum(num_right) / (m - i)
                info_gain = parent_entropy - (i / m) * self._entropy(classes[:i]) - (m - i) / m * self._entropy(classes[i:])
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)

        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
       
        return node.predicted_class

In [3]:
# Import thư viện và dữ liệu
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import pandas as pd
# Đọc dữ liệu
df = pd.read_csv("/content/car.csv")

# chuyển đổi dữ liệu về dạng số
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df['buying'] = label_encoder.fit_transform(df['buying'])
df['maint'] = label_encoder.fit_transform(df['maint'])
df['lug_boot'] = label_encoder.fit_transform(df['lug_boot'])
df['safety'] = label_encoder.fit_transform(df['safety'])
df['values'] = label_encoder.fit_transform(df['values'])
df['doors'] = label_encoder.fit_transform(df['doors'])
df['persons'] = label_encoder.fit_transform(df['persons'])

y = df['values'].values.reshape(-1,1)
X = df.drop(columns=['values'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tạo mô hình Decision Tree
dt = DecisionTreeClassifier()

# Huấn luyện mô hình trên tập huấn luyện
dt.fit(X_train, y_train)

#X_test = [[1, 1 ,3 ,2 ,1 ,2]] #good
#X_test = [[1, 1, 3 ,2 ,2 ,2]] #acc

# Dự đoán trên tập kiểm tra
y_pred = dt.predict(X_test)

# In kết quả dự đoán và độ chính xác trên tập kiểm tra
print("Predicted classes:", y_pred)
accuracy = np.mean(y_pred == y_test)
print("Độ chính xác của mô hình Decision Tree là: {:.2f}%".format(accuracy*100))

Predicted classes: [2 0 2 0 2 0 2 2 2 2 3 0 2 2 2 2 2 2 2 0 2 0 0 2 0 2 2 2 2 2 0 2 2 2 0 0 1
 2 2 2 2 2 0 0 1 0 2 2 2 2 2 2 0 2 2 2 2 2 2 1 2 0 2 2 0 3 2 0 0 2 2 2 2 2
 1 1 2 2 2 2 3 2 2 0 3 0 2 0 1 0 2 1 2 2 2 1 2 2 2 0 2 2 2 2 1 1 0 2 2 0 2
 2 2 2 2 0 1 2 2 2 2 0 2 0 0 2 1 2 2 2 2 2 0 2 2 0 2 3 2 2 2 0 0 0 2 0 2 2
 2 2 0 3 2 2 2 2 2 2 2 2 2 0 2 2 2 2 0 2 0 3 0 2 0 2 2 0 2 2 2 2 0 2 2 2 1
 0 2 2 0 2 2 2 0 2 2 2 2 0 2 2 2 2 2 1 2 2 2 1 2 2 2 2 0 2 2 0 2 2 2 1 3 2
 2 0 2 2 2 2 0 2 2 2 0 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0
 2 0 2 2 2 2 2 0 3 2 2 2 0 2 2 3 2 2 0 1 2 0 2 0 0 2 2 0 1 3 0 0 2 2 2 2 3
 0 0 2 2 2 2 3 2 2 2 2 0 2 2 3 0 0 2 2 0 0 0 2 0 0 2 2 2 2 2 2 0 2 2 2 2 0
 2 2 2 2 0 2 2 2 2 2 2 2 2]
Độ chính xác của mô hình Decision Tree là: 52.10%
